In [2]:
import torch 
import numpy 
dataset_name='ogbl-ddi'
import matplotlib
import matplotlib.pyplot as plt
from ogb.linkproppred import PygLinkPropPredDataset
import torch_geometric 
import myutils
dataset=PygLinkPropPredDataset(name=dataset_name)




Resources used for inspiration on code:
graph exploration from :
https://medium.com/mlearning-ai/ultimate-guide-to-graph-neural-networks-1-cora-dataset-37338c04fe6f

# Exploring our data

In [ ]:
data=dataset[0]
print(f'the {dataset_name} has {len(dataset)} graph')
print(f'number of nodes:{data.num_nodes}')
print(f'number of edges {data.num_edges}')
print(f'number of features {data.num_node_features}')
print(f'is data-graph directed? :{data.is_directed()}')
print(f'data has self-loops? : {data.has_self_loops()}')
print(f'data has isolated nodes? : {data.has_isolated_nodes()}')
print('the graph has average node degree of {:.2f}'.format(data.num_edges/data.num_nodes))

Data edges are given as two arrays 
array[0][i] holds the edge to array [1][i]
we look only on one array and infer the second
 

Helper functions

nx gets edges a stules


In [ ]:
#returns a tensor with the indices of neighbors of the node index
def get_neighbors(edge_index,node_index):
    edge_index=edge_index
    return edge_index[:,numpy.where(edge_index[0]==node_index)[0]][1]

import networkx as nx
def visualize_nx(edges_list):
    unique_list=numpy.unique(edges_list)
    print(f'the graph has {unique_list.shape} nodes')
    myGraph=nx.Graph()
    myGraph.add_nodes_from(unique_list)
    
    myGraph.add_edges_from(list(zip(edges_list[0],edges_list[1])))
    plt.figure()
    nx.draw_networkx(myGraph,with_labels=True)

    plt.show()




In [ ]:
edges2d_t=data.edge_index
node_example_t=edges2d_t[:,numpy.where(edges2d_t[0]==4)[0]]
node_example_n=node_example_t.numpy()[:,:5]
myGraph=nx.Graph()
myGraph.add_nodes_from(numpy.unique(node_example_n))
myGraph.add_edges_from(list(zip(node_example_n[0],node_example_n[1])))



nx.draw_networkx(myGraph,with_labels=False)

In [ ]:

node_example_t=edges2d_t[:,numpy.where(edges2d_t[0]==4)[0]].numpy()
node_example_t=node_example_t[:,:5]
node_example_t


In [ ]:
%%script echo skipping
myGraph=nx.Graph()
myGraph.add_nodes_from(data.edge_index[0])
myGraph.add_edges_from(list(zip(data.edge_index[0],data.edge_index[1])))

In [ ]:
import pandas 
def draw_degree_histogram(data):
    myGraph=nx.to_networkx_graph(list(zip(data[0].numpy(),data[1].numpy())))
    degrees=[val for (node,val) in myGraph.degree()]
    
    plt.hist(degrees,bins=range(0,max(degrees)+1))
    ax=plt.gca()
    plt.xlabel("# of interactions per drug (degree)")
    ax.set_ylim([0,30])
    plt.show()
    print(pandas.DataFrame(degrees).describe().transpose().round(3))



In [ ]:
draw_degree_histogram(data.edge_index)

In [ ]:
from pylab import rcParams
def draw_most_important(data):
    myGraph=nx.to_networkx_graph(list(zip(data[0].numpy(),data[1].numpy())))
    
    color_lookup={node:degree for node,degree in sorted(myGraph.degree())}
    print(color_lookup)

In [ ]:
myGraph=nx.to_networkx_graph(list(zip(data.edge_index[0].numpy(),data.edge_index[1].numpy())))
node_degree_sequence=numpy.array(object= sorted({(n,d) for (n,d) in myGraph.degree()},reverse=True,key=lambda x:x[1]))
node_degree_sequence[:10]

In [ ]:
low,high=node_degree_sequence[:,1].min(),node_degree_sequence[:,1].max()
print(f'low degree:{low}, high degree:{high}')

In [ ]:
node1=5
degreenode1=myGraph.degree(node1)
print(f'node {node1} has degree {degreenode1}')

nx.draw(
    G=myGraph,
    nodelist=[node1],
    node_color='red',
    with_labels=False,
)
plt.show()

In [ ]:
pos=nx.spring_layout(myGraph)
cent=nx.degree_centrality(myGraph)
node_size=list(map(lambda x:x*50,cent.values()))
cent_array=numpy.array(list(cent.values()))
threshold=sorted(cent_array,reverse=True)[10]
print(f'threshold:{threshold}')
cent_bin=numpy.where(cent_array>threshold,1,0.1)
plt.figure(figsize=(15,12))
nodes=nx.draw_networkx_nodes(
    G=myGraph,
    pos=pos,
    node_size=node_size,
    cmap=plt.cm.plasma,
    nodelist=list(cent.keys()),
    alpha=cent_bin,
    node_color=cent_bin

)
edges=nx.draw_networkx_edges(
    G=myGraph,
    pos=pos,
    width=0.03, alpha=0.2
)

## Get the sparse matrix version of the data.

We use ToSparseTensor to get a Tensor object with key adj_t

In [ ]:
dataset_sparse=PygLinkPropPredDataset(name='ogbl-ddi', transform=torch_geometric.transforms.ToSparseTensor())
device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
data_sparse=dataset_sparse[0]
adj_t=data_sparse.adj_t.to(device)
type(adj_t)

In [ ]:
split_edge=dataset.get_edge_split()
split_edge.items()

# Initial Features and Benchmark
## Topological similarity features
1. common neighbors
2. Jaccard's coefficient
3. Adamic/adar
4. Preferential attachment


In [ ]:
data=dataset[0]
Graph_total=nx.to_networkx_graph(list(zip(data.edge_index[0].numpy(), data.edge_index[1].numpy())))
pagerank_f=nx.pagerank(Graph_total, alpha=0.85)
clustering_coef_f=nx.clustering(Graph_total)
betweenness_f=nx.betweenness_centrality(Graph_total)
# adamic_adar_f=nx.adamic_adar_index(Graph_total)
# betweenness_f=nx.betweenness_centrality(Graph_total)




In [ ]:
common_neighbor_centrality_f=nx.common_neighbor_centrality(Graph_total)
save_to_txt("common_neighbor_centrality",common_neighbor_centrality_f)

In [ ]:

# feutures_emb=torch.ones(data.num_nodes,10, dtype=torch.float64).to(device)
# for _ in range(data.num_nodes):
#     features_emb[_][0]=pagerank_f[_]
#     features_emb[_][1]=clustering_coef_f[_]
#     features_emb[_][2]=betweenness_f[_]


In [ ]:
embedding_1s=torch.ones(data.num_nodes,dtype=torch.float64)
embedding_1s.to(device)

In [3]:

dataset=PygLinkPropPredDataset(name=dataset_name)
data=dataset[0]
split_edge=dataset.get_edge_split()
train_data=split_edge['train']
pagerank_f=myutils.get_dict_from_file("pagerank.txt")
clustering_coef_f=myutils.get_dict_from_file("clustering_coef.txt")
betweenness_f=myutils.get_dict_from_file("betweeness_centrality.txt")



In [4]:
from random import randint
m=len(train_data['edge'])
X_train=torch.zeros(2*len(train_data['edge']),3, dtype=torch.float64)
y_train=torch.zeros(2*len(train_data['edge']),1,dtype=torch.float64)
for x,edge in enumerate(train_data['edge']):
    X_train[x][0]=pagerank_f[edge[0].item()]+pagerank_f[edge[1].item()]
    X_train[x][1]=clustering_coef_f[edge[0].item()]+clustering_coef_f[edge[1].item()]
    X_train[x][2]=betweenness_f[edge[0].item()]+betweenness_f[edge[1].item()]

    y_train[x]=1

    random_node1=randint(0,data.num_nodes-1)
    random_node2=randint(0,data.num_nodes-1)
    X_train[m+x][0]=pagerank_f[random_node1]+pagerank_f[random_node2]
    X_train[m+x][1]=clustering_coef_f[random_node1]+clustering_coef_f[random_node2]
    X_train[m+x][2]=betweenness_f[random_node1]+betweenness_f[random_node2]

    y_train[m+x]=0
    #print("filling x at {0} %".format(x/m*100))

X_train.shape


    

torch.Size([2135822, 3])

In [11]:
test_data=split_edge['test']
X_test=torch.zeros(len(test_data['edge'])+len(test_data['edge_neg']),3, dtype=torch.float64)
y_test=torch.zeros(len(test_data['edge'])+len(test_data['edge_neg']),1,dtype=torch.float64)
for x,node in enumerate(test_data['edge']):
    X_test[x][0]=pagerank_f[node[0].item()]+pagerank_f[node[1].item()]
    X_test[x][1]=clustering_coef_f[node[0].item()]+clustering_coef_f[node[1].item()]
    X_test[x][2]=betweenness_f[node[0].item()]+betweenness_f[node[1].item()]
    y_test[x]=0

for x, node in enumerate(test_data['edge_neg']):
    X_test[len(test_data['edge'])+x][0]=pagerank_f[node[0].item()]+pagerank_f[node[1].item()]
    X_test[len(test_data['edge'])+x][1]=clustering_coef_f[node[0].item()]+clustering_coef_f[node[1].item()]
    X_test[len(test_data['edge'])+x][2]=betweenness_f[node[0].item()]+betweenness_f[node[1].item()]
    y_test[len(test_data['edge'])+x]=1
    

In [52]:
device='cuda' if torch.cuda.is_available() else 'cpu'
n_epochs=1000
loss_fn=torch.nn.BCELoss()
my_clf=LogisticRegressionModel(input_dim=3, output_dim=1)
X_train.to(device)
y_train.to(device)
optimizer=torch.optim.SGD(my_clf.parameters(), lr=0.01)
for epoch in range(n_epochs):
    y_pred=my_clf(X_train)
    loss=loss_fn(y_pred,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

Epoch: 0, Loss: 0.6931643683337694
Epoch: 1, Loss: 0.6931618587541306
Epoch: 2, Loss: 0.6931593752759058
Epoch: 3, Loss: 0.6931569176276727
Epoch: 4, Loss: 0.6931544855408295
Epoch: 5, Loss: 0.693152078749565
Epoch: 6, Loss: 0.6931496969908305
Epoch: 7, Loss: 0.6931473400043111
Epoch: 8, Loss: 0.6931450075323962
Epoch: 9, Loss: 0.6931426993201529
Epoch: 10, Loss: 0.6931404151152978
Epoch: 11, Loss: 0.6931381546681682
Epoch: 12, Loss: 0.6931359177316972
Epoch: 13, Loss: 0.693133704061385
Epoch: 14, Loss: 0.6931315134152727
Epoch: 15, Loss: 0.6931293455539164
Epoch: 16, Loss: 0.6931272002403605
Epoch: 17, Loss: 0.6931250772401123
Epoch: 18, Loss: 0.6931229763211161
Epoch: 19, Loss: 0.6931208972537283
Epoch: 20, Loss: 0.6931188398106914
Epoch: 21, Loss: 0.6931168037671114
Epoch: 22, Loss: 0.6931147889004304
Epoch: 23, Loss: 0.6931127949904046
Epoch: 24, Loss: 0.6931108218190788
Epoch: 25, Loss: 0.6931088691707641
Epoch: 26, Loss: 0.6931069368320133
Epoch: 27, Loss: 0.6931050245915973
Epoc

In [45]:
print(X_train)

tensor([[6.0906e-04, 1.3472e+00, 4.7768e-04],
        [9.1269e-04, 8.6194e-01, 4.1049e-03],
        [7.0281e-04, 1.0495e+00, 6.9128e-04],
        ...,
        [8.8032e-05, 5.7310e-01, 6.1928e-07],
        [2.9923e-04, 1.2672e+00, 4.8715e-05],
        [5.2902e-04, 5.4487e-01, 8.3086e-04]], dtype=torch.float64)


In [50]:
class LogisticRegressionModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim,dtype=torch.float64)

    def forward(self, inputs):
        x = self.linear(inputs)
        out=torch.sigmoid(x)
        return out